Importing library

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Importing Dataset

In [5]:
dataset = pd.read_csv("/content/drive/MyDrive/game_review_identification/train.csv")

Handling missing data

In [6]:
df = pd.DataFrame(dataset)
df = df.drop(columns=['review_id','title','year'])
print(df.shape)
df = df.dropna()
n = df.shape[0]
dataset = df

(17494, 2)


importing library to format text intu usable language processing

In [7]:
import re
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

processing text for tokenization

In [8]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus = []
for i in range(0,n):
  review = re.sub('[^a-zA-Z]', ' ', dataset['user_review'].iloc[i])
  review = review.lower()
  review = review.split()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

Counting vectors to exclude unimportant words

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 33000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

print(len(X[0]))


33000


splitting dataset into test and training set

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 1)

using random forest to train nlp model

In [11]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100,random_state =1 )
classifier.fit(X_train,y_train)

RandomForestClassifier(random_state=1)

predicting result with test set

In [12]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_test),1),y_test.reshape(len(y_test),1)),1))

[[1 0]
 [1 1]
 [1 1]
 ...
 [0 0]
 [0 0]
 [0 0]]


calcualting confusion matrix and accuracy score

In [13]:
from sklearn.metrics import confusion_matrix,accuracy_score
cfm = confusion_matrix(y_test,y_pred)
acs = accuracy_score(y_test,y_pred)
print(cfm)
print(acs)

[[1093  453]
 [ 185 1768]]
0.8176621891969134


stemming for custom review detection

In [14]:
def custom_result(s):
  custom_review = re.sub('[^a-zA-Z]', ' ', s)
  custom_review = custom_review.lower()
  custom_review = custom_review.split()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  ps = PorterStemmer()
  custom_review = [ps.stem(word) for word in custom_review if not word in set(all_stopwords)]
  custom_review = ' '.join(custom_review)
  return custom_review




This section is to take input for detecting the review if it is positive or negative

In [22]:
custom_review = input("Enter review: ")
custom_review = custom_result(custom_review)
custom_review = cv.transform([custom_review]).toarray()

if(classifier.predict(custom_review) == 1):
  print("Positive Review")
else:
  print("Negative Review")

#this is the best game ever
#totally worst game. waste of money.

Enter review: not good game at all
not good game
